# Laboratórios de Bioinformática
### Alinhamento múltiplo e filogenia

**Grupo 6**: Joana Gabriel, Maria Couto, Teresa Coimbra

Para efetuar o alinhamento múltiplo foram desenvolvidas algumas funções para automatizar o processo de junção das sequências a alinhar num só ficheiro fasta. Este processo foi utilizado para gerar os ficheiros do alinhamento das proteínas humanas. Para o ficheiro utilizado no alinhamento da proteína do vírus foram também adicionadas outras sequências que foram retiradas do SARS-CoV-2 *data hub* do NCBI.


A primeira função get_hit_id corre o blast e retorna uma lista com os IDS das sequências. Seguidamente, a função get_id recebe a lista gerada na função anterior e retorna apenas os ids (sem a formatação do alignment.hit_id que acrescenta caracteres, tais como 'sp' e '|'). Tal foi necessário para correr a função seguinte, que recebe a lista dos ids e cria um ficheiro fasta com as sequências correspondentes. 

In [1]:
def get_hit_id(filename, E_VALUE_THRES=0.0001, bit_score = 50, SCORE=200):
    ''' Corre o Blast e retorna uma lista com os IDS das sequências que resultaram dessa pesquisa'''
    from Bio.Blast import NCBIXML
    from Bio import SeqIO
    result_handle = open('./'+filename+'.xml') 
    blast_records = NCBIXML.read(result_handle)
    
    List1 = []
    for alignment in blast_records.alignments:
        for hsp in alignment.hsps:
            if (hsp.expect < E_VALUE_THRES) and (hsp.bits > bit_score) and (hsp.score > SCORE):
                List1.append(alignment.hit_id)
        
    return List1

In [2]:
def get_id(hit_id_list):
    '''Recebe uma lista de Id de cada hit e retorna apenas os ids'''
    id_list = []
    for i in hit_id_list:
        ids = i.split('|')
        id_list.append(ids[1])
    return id_list

In [3]:
def get_fasta_file(ids, nome):
    '''Recebe uma lista de ids para um dos genes e o nome a dar ao ficheiro
    Cria um ficheiro fasta com o conjunto das sequências que correspondem aos ids'''
    from Bio import Entrez
    from Bio import SeqIO
    import re
    record_list = []
    for id0 in ids:
        Entrez.email = 'lbiogrupo6@gmail.com'
        handle = Entrez.efetch(db="protein", id=id0, rettype="fasta", retmode="text")
        record = SeqIO.read(handle, "fasta")
        record_list.append(record)
    SeqIO.write(record_list, nome+'.faa', 'fasta')

### HNRNP A1 - heterogeneous nuclear ribonucleoprotein A1-like 2 

In [13]:
from Bio.Seq import Seq
from Bio.SeqRecord import SeqRecord
from Bio.Align import MultipleSeqAlignment
from Bio import AlignIO

In [9]:
# hnrnp = get_hit_id('NP_001011725.1')
# hnrnp_id = get_id(hnrnp)
# get_fasta_file(hnrnp_id, 'nucleoprotein_3')

In [19]:
al1 = AlignIO.read('HNRNPA1.fas','fasta')
print(al1)

Alignment with 18 rows and 429 columns
---------------------MSKSESPKEPEQLRKLFIGGLSF...F-- NP002127.1|HNRNP
---------------------MSKSESPKEPEQLRKLFIGGLSF...F-- NP112420.1|HNRNP
---------------------MSKSASPKEPEQLRKLFIGGLSF...FQ- NP001011724.1|HNRNP
---------------------MSKSASPKEPEQLRKLFIGGLSF...FQ- NP001011725.1|HNRNP
---------------------MSKSASPKEPEQLRKLFIGGLSF...FQ- Q32P51.2|RA1L2
---------------------MSKSESPKEPEQLRKLFIGGLSF...FQ- A5A6H4.1|ROA1
---------------------MSKSESPKEPEQLRKLFIGGLSF...FQ- P04256.3|ROA1
---------------------MSKSESPKEPEQLRKLFIGGLSF...FQ- Q28521.3|ROA1
---------------------MSKSESPKEPEQLRKLFIGGLSF...F-- P09651.5|ROA1
---------------------MHKSEAPNEPEQLRKLFIGGLSF...FY- P17130.1|ROA1
--------MPRGGMDDHWPSSDDQGHEPKEPEQLRKLFIGGLSF...FY- P51992.1|RO32
MEVKPPPGRPQPDSGRRRRRRGEEGHDPKEPEQLRKLFIGGLSF...--- P51991.2|ROA3
MEVKPPPGRPQPDSGRRRRRRGEEGHDPKEPEQLRKLFIGGLSF...--- Q6URK4.1|ROA3
--------------------------MEREKEQFRKLFIGGLSF...RYM P51989.1|RO21
--------------------------MEREKE

### Converta esse ficheiro para o formato stockholm.
Através do formato PFAM/Stockholm é possível adicionar anotação que mostre a qualidade do alinhamento.

In [20]:
from Bio import AlignIO
al_st = AlignIO.convert("HNRNPA1.fas",'fasta',"HNRNPA1.sth",'stockholm')
print("Converted %i alignments" % al_st)
print(al_st)

Converted 1 alignments
1


In [21]:
alignment = AlignIO.read("HNRNPA1.sth", "stockholm")
for record in alignment:
    print("%s - %s" % (record.seq, record.id))
print('\nAlinhamento:', alignment)

---------------------MSKSESPKEPEQLRKLFIGGLSFETTDESLRSHFEQWGTLTDCVVMRDPNTKRSRGFGFVTYATVEEVDAAMNARPHKVDGRVVEPKRAVSREDSQRPGAHLTVKKIFVGGIKEDTEEHHLRDYFEQYGKIEVIEIMTDRGSGKKRGFAFVTFDDHDSVDKIVIQKYHTVNGHNCEVRKALSKQEMASASSSQRGRSGSGNFG----------GGRGGGFGGNDNFGRGGNFSGRGG---------------------FGGSRGGGGYGGSGDGYNGFGNDG----------------------------------------------------SNFGGGGSYNDFGNYNNQSS-NFGPMKGG-NFGG-RSSGPYGGGGQYFAKPRNQGGYGGSSSSSSYGSGRRF-- - NP002127.1|HNRNP
---------------------MSKSESPKEPEQLRKLFIGGLSFETTDESLRSHFEQWGTLTDCVVMRDPNTKRSRGFGFVTYATVEEVDAAMNARPHKVDGRVVEPKRAVSREDSQRPGAHLTVKKIFVGGIKEDTEEHHLRDYFEQYGKIEVIEIMTDRGSGKKRGFAFVTFDDHDSVDKIVIQKYHTVNGHNCEVRKALSKQEMASASSSQRGRSGSGNFG----------GGRGGGFGGNDNFGRGGNFSGRGG---------------------FGGSRGGGGYGGSGDGYNGFGNDGGYGGGGPGYSGGSRGYGSGGQGYGNQGSGYGGSGSYDSYNNGGGGGFGGGSGSNFGGGGSYNDFGNYNNQSS-NFGPMKGG-NFGG-RSSGPYGGGGQYFAKPRNQGGYGGSSSSSSYGSGRRF-- - NP112420.1|HNRNP
---------------------MSKSASPKEPEQLRKLFIGGLSFETTDESLRSHFEQWGTLTDCVVMRDPNTKRSRGFGFVTYATVEEVDAAMNTTPHKVDG

In [44]:
from Bio import Phylo
tree_UPGMA1 =Phylo.read('./hnrnpa1.nwk', 'newick')
#print (tree_UPGMA1)
print('FILOGENIA HNRNPA1 UPGMA')
Phylo.draw_ascii(tree_UPGMA1)

FILOGENIA HNRNPA1 UPGMA
                      , A5A6H4.1_|_ROA1_PANTR_|_Pan_troglodytes
                      |
                      | P09651.5|ROA1_HUMAN_|_Homo_sapiens
                      |
                      | NP_112420.1_|_HNRNP_A1_isoform_b_|_Ho...
                      |
                      | NP_002127.1_|_HNRNP_A1_isoform_a_|_Ho...
                      |
                     _| P04256.3_|_ROA1_RAT_|_Rattus_nordvegicus
                    | |
          __________| | Q28521.3_|_ROA1_MACMU_|_Macaca_mulatta
         |          |
         |          | , NP_001011724.1_|_HNRNP_A1-like_2_|_Ho...
         |          |_|
  _______|            , NP_001011725.1_|_HNRNP_A1-like_2_|_Ho...
 |       |            |
 |       |            | Q32P51.2_|_RA1L2_HUMAN_|_Homo_sapiens
 |       |
 |       |____________ P17130.1|ROA1_XENLA_|_Xaenopus_laevis
 |
 |          __________ P51992.1|RO32_XENLA_|_Xaenopus_laevis
 |_________|
_|         |          , P51991.2|ROA3_HUMAN_|_Homo_sapiens
 |   

In [45]:
from Bio import Phylo
tree_ML1 =Phylo.read('./hnrnpa1_ml.nwk', 'newick')
#print (tree_ML1)
print('FILOGENIA HNRNPA1 MAXIMUM LIKELIHOOD')
Phylo.draw_ascii(tree_ML1)

FILOGENIA HNRNPA1 MAXIMUM LIKELIHOOD
                       , NP_002127.1_|_HNRNP_A1_isoform_a_|_Ho...
                       |
                       | P09651.5|ROA1_HUMAN_|_Homo_sapiens
                       |
                       | NP_112420.1_|_HNRNP_A1_isoform_b_|_Ho...
                       |
                       | P04256.3_|_ROA1_RAT_|_Rattus_nordvegicus
                       |
                       | , NP_001011724.1_|_HNRNP_A1-like_2_|_Ho...
                       |_|
                       | , NP_001011725.1_|_HNRNP_A1-like_2_|_Ho...
                       | |
                       | | Q32P51.2_|_RA1L2_HUMAN_|_Homo_sapiens
             __________|
            |          | A5A6H4.1_|_ROA1_PANTR_|_Pan_troglodytes
  __________|          |
 |          |          | Q28521.3_|_ROA1_MACMU_|_Macaca_mulatta
 |          |
 |          |__________ P17130.1|ROA1_XENLA_|_Xaenopus_laevis
 |
 |             _______ P51992.1|RO32_XENLA_|_Xaenopus_laevis
_|____________|
 |            |

### SMAD 3 - mothers against decapentaplegic homolog 3 isoform 1

In [6]:
# smad = get_hit_id('NP_005893.1')
# smadid = get_id(smad)
# get_fasta_file(smadid, 'smad_3')

In [25]:
al2 = AlignIO.read('smad3_for_align.fas','fasta')
print(al2)

Alignment with 18 rows and 526 columns
----------------MSSILPFTPPIVKRLLGWKKGE------...VS- NP
--------------------------------------------...VSN NP001138574.1|
--------------------------------------------...VSP NP001138576.1|
----------------MSSILPFTPPIVKRLLGWKKGE------...VS- P84022.1|SMAD3|Homo
----------------MSSILPFTPPIVKRLLGWKKGE------...VS- P84023.1
----------------MSSILPFTPPVVKRLLGWKKSAGGSGGA...MS- Q62432.2|SMAD2
----------------MSSILPFTPPVVKRLLGWKKSAGGSGGA...MS- Q15796.1|SMAD2
----------------MSSILPFTPPVVKRLLGWKKSAGGSGGA...MS- Q5R7C0.1|SMAD2
----------------MSSILPFTPPVVKRLLGWKKSAGGSGGA...MS- O70436.1|SMAD2
----------------MSSILPFTPPVVKRLLGWKKSAGGSGGA...MS- Q1W668.1|SMAD2
----------------MSSILPFTPPVVKRLLGWKKSASGSSGA...MS- Q9I9P9.1|SMAD2
-------------MTSMASLFSFTSPAVKRLLGWKQGD------...VSS P97454.2|SMAD5
-------------MTSMASLFSFTSPAVKRLLGWKQGD------...VSS Q56I99.1|SMAD5
-------------MTSMASLFSFTSPAVKRLLGWKQGD------...VSS Q5R6H7.1|SMAD5
-------------MTSMSSLFSFTSPAVKRLLGWKQGD------...VSS

In [26]:
from Bio import AlignIO
al_smad3= AlignIO.convert("smad3_for_align.fas",'fasta',"smad3_for_align.sth",'stockholm')
print("Converted %i alignments" % al_st)
print(al_smad3)

Converted 1 alignments
1


In [27]:
alignment2 = AlignIO.read("smad3_for_align.sth", "stockholm")
for record in alignment2:
    print("%s - %s" % (record.seq, record.id))
print('\nAlinhamento:', alignment2)

---------------------MSKSESPKEPEQLRKLFIGGLSFETTDESLRSHFEQWGTLTDCVVMRDPNTKRSRGFGFVTYATVEEVDAAMNARPHKVDGRVVEPKRAVSREDSQRPGAHLTVKKIFVGGIKEDTEEHHLRDYFEQYGKIEVIEIMTDRGSGKKRGFAFVTFDDHDSVDKIVIQKYHTVNGHNCEVRKALSKQEMASASSSQRGRSGSGNFG----------GGRGGGFGGNDNFGRGGNFSGRGG---------------------FGGSRGGGGYGGSGDGYNGFGNDG----------------------------------------------------SNFGGGGSYNDFGNYNNQSS-NFGPMKGG-NFGG-RSSGPYGGGGQYFAKPRNQGGYGGSSSSSSYGSGRRF-- - NP002127.1|HNRNP
---------------------MSKSESPKEPEQLRKLFIGGLSFETTDESLRSHFEQWGTLTDCVVMRDPNTKRSRGFGFVTYATVEEVDAAMNARPHKVDGRVVEPKRAVSREDSQRPGAHLTVKKIFVGGIKEDTEEHHLRDYFEQYGKIEVIEIMTDRGSGKKRGFAFVTFDDHDSVDKIVIQKYHTVNGHNCEVRKALSKQEMASASSSQRGRSGSGNFG----------GGRGGGFGGNDNFGRGGNFSGRGG---------------------FGGSRGGGGYGGSGDGYNGFGNDGGYGGGGPGYSGGSRGYGSGGQGYGNQGSGYGGSGSYDSYNNGGGGGFGGGSGSNFGGGGSYNDFGNYNNQSS-NFGPMKGG-NFGG-RSSGPYGGGGQYFAKPRNQGGYGGSSSSSSYGSGRRF-- - NP112420.1|HNRNP
---------------------MSKSASPKEPEQLRKLFIGGLSFETTDESLRSHFEQWGTLTDCVVMRDPNTKRSRGFGFVTYATVEEVDAAMNTTPHKVDG

In [46]:
from Bio import Phylo
tree_UPGMA2 =Phylo.read('./smad_3.nwk', 'newick')
#print (tree_UPGMA2)
print('FILOGENIA SMAD3 UPGMA')
Phylo.draw_ascii(tree_UPGMA2)

FILOGENIA SMAD3 UPGMA
    , Q15796.1|SMAD2_HUMAN_|_Homo_sapiens
    |
    | Q1W668.1|SMAD2_BOVIN_MAD_|_Bos_taurus
    |
   _| Q5R7C0.1|SMAD2_PONAB_|_Pongo_abelii
  | |
  | , Q62432.2|SMAD2_MOUSE|_Mus_musculus
 ,| |
 || | O70436.1|SMAD2_RAT|_Rattus_norvegicus
 ||
 ||_ Q9I9P9.1|SMAD2_DANRE_|_Danio_rerio
 |
 |  , P84023.1_|SMAD3_CHICK|_Gallus_gallus
 |__|
 |  , NP_001138574.1|_SMAD3_isoform_3_|_Hom...
 |  |
_|  , NP_005893.1_|_SMAD3_isoform_1_|_Homo_...
 |  |
 |  , NP_001138576.1|_SMAD3_isoform_4_|_Hom...
 |  |
 |  | P84022.1|SMAD3|Homo_sapiens
 |
 |                      __________ P42003.1|MAD_DROME_|_Drosophila_melan...
 |_____________________|
                       |     _____ Q15797.1|SMAD1_HUMAN|_Homo_sapiens
                       |____|
                            |   ___ Q9W7E7.1|SMAD5_DANRE_|_Danio_rerio
                            |__|
                               | , Q56I99.1|SMAD5_CHICK_|_Gallus_gallus
                               |_|
                                 |, P

In [47]:
from Bio import Phylo
tree_ML2 =Phylo.read('./smad3_ml.nwk', 'newick')
#print (tree_ML2)
print('FILOGENIA SMAD3 MAXIMUM LIKELIHOOD')
Phylo.draw_ascii(tree_ML2)

FILOGENIA SMAD3 MAXIMUM LIKELIHOOD
       , NP_005893.1_|_SMAD3_isoform_1_|_Homo_...
       |
      ,| NP_001138574.1|_SMAD3_isoform_3_|_Hom...
      ||
      || P84022.1|SMAD3|Homo_sapiens
      |
   ___| P84023.1_|SMAD3_CHICK|_Gallus_gallus
  |   |
  |   | NP_001138576.1|_SMAD3_isoform_4_|_Hom...
 ,|
 ||, Q62432.2|SMAD2_MOUSE|_Mus_musculus
 |||
 ||| O70436.1|SMAD2_RAT|_Rattus_norvegicus
 |||
 | , Q5R7C0.1|SMAD2_PONAB_|_Pongo_abelii
 | |
_| , Q15796.1|SMAD2_HUMAN_|_Homo_sapiens
 | |
 | | Q1W668.1|SMAD2_BOVIN_MAD_|_Bos_taurus
 |
 | Q9I9P9.1|SMAD2_DANRE_|_Danio_rerio
 |
 |                    _______ P42003.1|MAD_DROME_|_Drosophila_melan...
 |___________________|
                     |      _ Q9W7E7.1|SMAD5_DANRE_|_Danio_rerio
                     |_____|
                           | ______ Q15797.1|SMAD1_HUMAN|_Homo_sapiens
                           ||
                            |, Q56I99.1|SMAD5_CHICK_|_Gallus_gallus
                            ||
                             |, Q9R1

### Nucleoprotein 

In [7]:
# np = get_hit_id('P0DTC9.1')
# np_id = get_id(np)
# get_fasta_file(np_id, 'nucleoprotein_3')

### Verificação dos alinhamentos com o AlignIO.readIO

In [30]:
al3 = AlignIO.read('nucleoprotein.fas','fasta')
print(al3)

Alignment with 15 rows and 504 columns
----MSDNGPQNQRNAPRITFGGPSDSTGS---NQNGERSGARS...--- P0DTC9.1
----MSDNGPQNQRNAPRITFGGPSDSTGS---NQNGERSGARS...--- QOD07317.1
----MSDNGPQNQRNAPRITFGGPSDSTGS---NQNGERSGARS...--- QQV26719.1
----MSDNGPQNQRNAPRITFGGPSDSTGS---NQNGERSGARS...--- QQZ90878.1
----MSDNGPQNQRNAPRITFGGPSDSTGS---NQNGERSGARS...--- YP
----MSDNGPQNQRNAPRITFGGPSDSTGS---NQNGERSGARS...--- QOT48517.1
----MSDNGPQNQRSAPRITFGGPTDSTDN---NQDGGRSGARP...--- Q3I5I7.1
---MSGRNRSRSGTPSPKVTFKQESDGSDSESERRNGNRNGARP...ERS A3EXH0.1
MSYTPG-HHAGSRSSSGNRS--GILKKTSWVDQSERSHQTYNRG...--- Q0ZME3.1
MSFVPGQENAGSRSSSGNRAGNGILKKTTWADQTERGLNNQNRG...--- P03417.1
MSFVPGQENAGSRSSSGNRAGNGILKKTTWADQTERGPNNQNRG...--- Q83360.1
MSFVPGQENAGSRSSSGNRAGNGILKKTTWADQTERGSNNQNRG...--- P59713.1
----------MASPAAPRAVSFA--DNND----ITNTNLSRGRG...--- QJX19964.1
----------MASPAAPRAVSFA--DNND----ITNTNLSRGRG...--- QFN25227.1
----------MASPAAPRAVSFA--DNND----ITNTNLSRGRG...--- QGV13487.1


In [40]:
from Bio import AlignIO
al_nucl = AlignIO.convert("nucleoprotein.fas",'fasta',"nucleoprotein.sth",'stockholm')
print("Converted %i alignments" % al_nucl)
print(al_nucl)

Converted 1 alignments
1


In [41]:
alignment3 = AlignIO.read("nucleoprotein.sth", "stockholm")
for record in alignment3:
    print("%s - %s" % (record.seq, record.id))
print('\nAlinhamento:', alignment3)

----MSDNGPQNQRNAPRITFGGPSDSTGS---NQNGERSGARSKQ----------RRPQGLPNNTASWFTALTQHGK-EDLKFPRGQGVPINTNSSPDDQIGYYRRATR-RIRGGDGKMKDLSPRWYFYYLGTGPEAGLPYGANKDGIIWVATEGALNTPKDHIGTRNPANNAAIVLQLPQGTTLPKGFYAEGSRGGSQASSRSSSRSRNSSRNSTPGSSRGTSP------ARMAGNGGDAALALLLLDRLNQLESKMSG-KGQQQQGQTVTKKSAAEASKKPRQKRTATKAYNVTQAFGRRGPEQTQGNFGDQELIRQGTDYKHWPQIAQFAPSASAFFGMSRIGMEVTPSG---------TWLTYTGAIKLDDKDPNFKDQVILLNKHIDAYKT----------FPPTEPKKDKKKK-------------ADETQALPQRQKKQQTVTLLPAADLDDFSKQLQQSMSSADSTQA--------------------------- - P0DTC9.1
----MSDNGPQNQRNAPRITFGGPSDSTGS---NQNGERSGARSKQ----------RRPQGLPNNTASWFTALTQHGK-EDLKFPRGQGVPINTNSSPDDQIGYYRRATR-RIRGGDGKMKDLSPRWYFYYLGTGPEAGLPYGANKDGIIWVATEGALNTPKDHIGTRNPANNAAIVLQLPQGTTLPKGFYAEGSRGGSQASSRSSSRSRNSSRNSTPGSSRGTSP------ARMAGNGGDAALALLLLDRLNQLESKMSG-KGQQQQGQTVTKKSAAEASKKPRQKRTATKAYNVTQAFGRRGPEQTQGNFGDQELIRQGTDYKHWPQIAQFAPSASAFFGMSRIGMEVTPSG---------TWLTYTGAIKLDDKDPNFKDQVILLNKHIDAYKT----------FPPTEPKKDKKKK-------------ADETQALPQRQKKQQTVTLLPAADLDDFSKQLQQSMSSADSTQA-------

In [50]:
from Bio import Phylo
tree_UPGMA3 =Phylo.read('./nucleoprotein_2.nwk', 'newick')
#print (tree_UPGMA3)
print('FILOGENIA Protein N UPGMA')
Phylo.draw_ascii(tree_UPGMA3)

FILOGENIA Protein N UPGMA
                        , QOD07317.1_|_N_protein_|_SARS-CoV-2_|...
                        |
                        | YP_009724397.2_|_N_protein_|_SARS-CoV...
                        |
                        | P0DTC9.1_|_N_protein_|_SARS-CoV-2
                        |
                      __| QQV26719.1_|_N_protein_|_SARS-CoV-2_|...
                     |  |
                     |  , QQZ90878.1_|_N_protein_|_SARS-CoV-2_|...
      _______________|  |
     |               |  | QOT48517.1_|_N_protein_|_SARS-CoV-2_|...
     |               |
  ___|               |__ Q3I5I7.1_|_N_protein_|_Bat_SARS_CoV
 |   |
 |   |                  , QFN25227.1_|_N_protein_|_MERS_|Saudi_...
 |   |__________________|
 |                      , QJX19964.1_|_N_protein_|_MERS_|_South...
 |                      |
 |                      | QGV13487.1_|_N_protein_|_MERS_|_Egypt...
_|
 |______________________ A3EXH0.1_|_N_protein_|_Rousettus_bat_...
 |
 |                         ______

In [51]:
from Bio import Phylo
tree_ML3 =Phylo.read('./nprot_max_lik.nwk', 'newick')
#print (tree_ML3)
print('FILOGENIA Proteína N MAXIMUM LIKELIHOOD')
print('\n')
Phylo.draw_ascii(tree_ML3)

FILOGENIA Proteína N MAXIMUM LIKELIHOOD


                    , P0DTC9.1_|_N_protein_|_SARS-CoV-2
                    |
                    | QOD07317.1_|_N_protein_|_SARS-CoV-2_|...
                    |
                    | YP_009724397.2_|_N_protein_|_SARS-CoV...
                    |
                    , QQZ90878.1_|_N_protein_|_SARS-CoV-2_|...
                   ,|
                   || QOT48517.1_|_N_protein_|_SARS-CoV-2_|...
        ___________||
       |           || QQV26719.1_|_N_protein_|_SARS-CoV-2_|...
       |           |
  _____|           |__ Q3I5I7.1_|_N_protein_|_Bat_SARS_CoV
 |     |
 |     |               , QFN25227.1_|_N_protein_|_MERS_|Saudi_...
 |     |_______________|
 |                     , QJX19964.1_|_N_protein_|_MERS_|_South...
_|                     |
 |                     | QGV13487.1_|_N_protein_|_MERS_|_Egypt...
 |
 |__________________ A3EXH0.1_|_N_protein_|_Rousettus_bat_...
 |
 |                           ______ Q0ZME3.1_|_N_protein_|_HKU1_Human_co